# How to: Find and Access EMIT Data

**Summary**  

There are currently 3 ways to find EMIT data:

1. [EarthData Search](https://search.earthdata.nasa.gov/search)
2. [NASA's CMR API](https://www.earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr)
3. [Visions Open Access Data Portal](https://earth.jpl.nasa.gov/emit/data/data-portal/coverage-and-forecasts/)

This notebook will explain how to access Earth Surface Minteral Dust Source Investigation (EMIT) data programmaticly using the [earthaccess python library](https://github.com/nsidc/earthaccess). `earthaccess` is an easy to use library that reduces finding and downloading or streaming data over https or s3 to only a few lines of code. `earthaccess` searches NASA's Common Metadata Repository (CMR), a metadata system that catalogs Earth Science data and associated metadata records, then can be used to download granules or generate lists granule search result URLs.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**  
- How to search and access EMIT data using `earthaccess`

---

Import the required packages

In [1]:
import os
import earthaccess
import pandas as pd
import geopandas as gp
import xarray as xr
import sys
sys.path.append('../modules/')
from emit_tools import emit_xarray


## Authentication

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [2]:
earthaccess.login(persist=True)

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 07/08/2023
Using .netrc file for EDL


---

## Searching for Collections

If we want to see the available EMIT collections, we can 


In [3]:
Query = earthaccess.collection_query().keyword('emit').provider('LPCLOUD')
print(f'Collections found: {Query.hits()}')

Collections found: 4


We can retrieve metadata for these collections, and then the shortnames so we can search for granules. 

In [4]:
# Find Collections
collections = Query.fields(['ShortName']).get(4)
# Retrieve Collection Short-names
[product['short-name'] for product in [collection.summary() for collection in collections]]

['EMITL2ARFL', 'EMITL1BRAD', 'EMITL2BMIN', 'EMITL1BATT']

If you print the `collections` object you can explore all of the json metadata.

---
## Searching for Granules

A granule can be thought of as a unique spatiotemporal grouping within a collection. To search for granules, we simply use the `search_data` function from `earthaccess` and provide the arguments for our search. Its possible to specify search products using several criteria shown in the table below:

|dataset origin and location|spatio temporal parameters|dataset metadata parameters|
|:---|:---|:---|
|archive_center|bounding_box|concept_id
|data_center|temporal|entry_title
|daac|point|keyword
|provider|polygon|version
|cloud_hosted|line|short_name

### Point Search

In this case, we specify the `shortname`, `point`, and `temporal`, as well as `count`, which limits the maximum number of results returned. 

In [5]:
# POINT
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    point=(-62.1123,-39.89402),
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

Granules found: 1


### Bounding Box Search

You can also use a bounding box to search. To do this we will first open a geojson file containing our region of interest (ROI) then simplify it to a bounding box by getting the bounds and putting them into a tuple. We will use the `total_bounds` property to get the bounding box of our ROI, and add that to a python tuple, which is the expected data type for the bounding_box parameter `earthaccess` `search_data`.

In [6]:
geojson = gp.read_file('../../data/isla_gaviota.geojson')
geojson.geometry

0    POLYGON ((-62.14758 -39.88951, -62.16900 -39.8...
Name: geometry, dtype: geometry

In [7]:
bbox = tuple(list(geojson.total_bounds))
bbox

(-62.20427143422259,
 -39.95230375907932,
 -62.11609022486114,
 -39.87693893067732)

Now we can search for granules using the a bounding box.

In [8]:
# Search Example using Bounding Box
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    bounding_box=bbox,
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

Granules found: 1



### Polygon Search

A polygon can also be used to search. For a simple polygon without holes we can take the geojson we opened and grab the coordinates of the exterior ring and place them in a list.

In [9]:
polygon = list(geojson.geometry[0].exterior.coords)
polygon

[(-62.147583513919045, -39.88950549416461),
 (-62.16899895047814, -39.87693893067732),
 (-62.19419358172446, -39.90641838472922),
 (-62.20427143422259, -39.94071456822524),
 (-62.1318368693898, -39.95230375907932),
 (-62.11609022486114, -39.92091182572591),
 (-62.125538211578245, -39.895787912197314),
 (-62.147583513919045, -39.88950549416461)]

With this list of coordinate pairs we can use the `polygon` parameter for our search. 
> Note that we overwrote the `results` object, because for all 3 types spatial search, the `results` are the same for this example.

In [10]:
# Search Example using a Polygon
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    polygon=polygon,
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

Granules found: 1


---

## Working with Search Results

After we've gotten results from our search using `earthaccess` we can view the results in a table and view assets for each granule in the list. 

In [11]:
pd.json_normalize(results)

,size,meta.concept-type,meta.concept-id,meta.revision-id,meta.native-id,meta.provider-id,meta.format,meta.revision-date,umm.TemporalExtent.RangeDateTime.BeginningDateTime,umm.TemporalExtent.RangeDateTime.EndingDateTime,...,umm.PGEVersionClass.PGEVersion,umm.RelatedUrls,umm.CloudCover,umm.DataGranule.DayNightFlag,umm.DataGranule.ArchiveAndDistributionInformation,umm.DataGranule.ProductionDateTime,umm.Platforms,umm.MetadataSpecification.URL,umm.MetadataSpecification.Name,umm.MetadataSpecification.Version
0,0,granule,G2544480896-LPCLOUD,3,EMIT_L2A_RFL_001_20220903T163129_2224611_012,LPCLOUD,application/vnd.nasa.cmr.umm+json,2023-03-20T22:08:19.486Z,2022-09-03T16:31:29Z,2022-09-03T16:31:41Z,...,v1.3.3,[{'URL': 'https://opendap.earthdata.nasa.gov/c...,82,Day,[{'Name': 'EMIT_L2A_RFL_001_20220903T163129_22...,2023-03-20T21:50:48Z,"[{'ShortName': 'ISS', 'Instruments': [{'ShortN...",https://cdn.earthdata.nasa.gov/umm/granule/v1.6.4,UMM-G,1.6.4


In [12]:
results[0]

Collection: {'ShortName': 'EMITL2ARFL', 'Version': '001'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -62.08873748779297, 'Latitude': -39.2420539855957}, {'Longitude': -62.51200866699219, 'Latitude': -39.943302154541016}, {'Longitude': -61.660133361816406, 'Latitude': -40.45749282836914}, {'Longitude': -61.23686218261719, 'Latitude': -39.75624465942383}, {'Longitude': -62.08873748779297, 'Latitude': -39.2420539855957}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2022-09-03T16:31:29Z', 'EndingDateTime': '2022-09-03T16:31:41Z'}}
Size(MB): 0
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611_012.nc', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc']

After we have our results, there are 2 ways we an work with the data:

1. Download
2. Access in place / Stream the data. 

To download the data we can simply use the download function. This will retrieve all assets associated with a granule, and is nice if you plan to work with the data in this way.

In [13]:
earthaccess.download(results, '../../data/')

 Getting 1 granules, approx download size: 0.0 GB


QUEUEING TASKS | :   0%|          | 0/3 [00:00<?, ?it/s]

File EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc already downloaded
File EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611_012.nc already downloaded
File EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc already downloaded


PROCESSING TASKS | :   0%|          | 0/3 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/3 [00:00<?, ?it/s]

['EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc',
 'EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611_012.nc',
 'EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc']

If we want to stream the data or further filter the assets for download we want to first create a list of URLs nested by granule using list comprehesion.

In [14]:
emit_results_urls = [granule.data_links() for granule in results]
emit_results_urls

[['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFLUNCERT_001_20220903T163129_2224611_012.nc',
  'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_MASK_001_20220903T163129_2224611_012.nc']]

Now we can also split these into results for specific assets or filter out an asset using the following. In this example, we only want to access or download reflectance.

In [15]:
filtered_asset_links = []
# Pick Desired Assets (leave _ on RFL to distinguish from RFLUNC)
desired_assets = ['RFL_'] # Add more or do individually for reflectance, reflectance uncertainty, or mask
# Step through each sublist (granule) and filter based on desired assets.
for n, granule in enumerate(emit_results_urls):
    for url in granule: 
        asset_name = url.split('/')[-1]
        if any(asset in asset_name for asset in desired_assets):
            filtered_asset_links.append(url)
filtered_asset_links

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc']

After we have our filtered list, we can stream the reflectance asset or download it. Start an https session then open it to stream the data, or download to save the file.

#### Stream Data

This may take a while to load the dataset.

In [16]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
url = filtered_asset_links[0]
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = fs.open(url)
# Open with `emit_xarray` function
ds = emit_xarray(fp)
ds

<xarray.Dataset>
Dimensions:           (downtrack: 1280, crosstrack: 1242, wavelengths: 285,
                       ortho_y: 2009, ortho_x: 2353)
Coordinates:
  * downtrack         (downtrack) int64 0 1 2 3 4 5 ... 1275 1276 1277 1278 1279
  * crosstrack        (crosstrack) int64 0 1 2 3 4 ... 1237 1238 1239 1240 1241
    lon               (downtrack, crosstrack) float64 ...
    lat               (downtrack, crosstrack) float64 ...
    elev              (downtrack, crosstrack) float64 ...
    glt_x             (ortho_y, ortho_x) float64 ...
    glt_y             (ortho_y, ortho_x) float64 ...
  * wavelengths       (wavelengths) float32 381.0 388.4 ... 2.486e+03 2.493e+03
    fwhm              (wavelengths) float32 ...
    good_wavelengths  (wavelengths) float32 ...
Dimensions without coordinates: ortho_y, ortho_x
Data variables:
    reflectance       (downtrack, crosstrack, wavelengths) float32 0.02187 .....
Attributes: (12/39)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...

#### Download Filtered 

In [17]:
# Get Https Session using Earthdata Login Info
fs = earthaccess.get_fsspec_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
for url in filtered_asset_links:
    granule_asset_id = url.split('/')[-1]
    # Define Local Filepath
    fp = f'../../data/{granule_asset_id}'
    # Download the Granule Asset if it doesn't exist
    if not os.path.isfile(fp):
        fs.download(url, fp)

---

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 07-03-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I. 